In [1]:
SOURCE = '../data/caterpillar/caterpillar-tube-pricing/competition_data/'

In [10]:
import os;
import numpy as np;
import pandas as pd;
from glob import glob;
from sklearn.metrics import mean_squared_error, mean_squared_log_error;

In [11]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred));
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred));

In [6]:
data = {};
for path in glob(os.path.join(SOURCE, '*.csv')):
    df = pd.read_csv(path);
    fname = path.split(SOURCE)[1];
    name = fname.split('.csv')[0];
    data[name] = df;

In [13]:
materials = data['bill_of_materials'].copy();
materials.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
components = data['components'].copy();
components.head()

,component_id,name,component_type_id
0,9999,OTHER,OTHER
1,C-0001,SLEEVE,CP-024
2,C-0002,SLEEVE,CP-024
3,C-0003,SLEEVE-FLARED,CP-024
4,C-0004,NUT,CP-026


In [19]:
assembly_components = materials.melt(id_vars='tube_assembly_id',
                                     value_vars=[f'component_id_{n}' for n in range(1, 9)]);

assembly_components = (assembly_components
                      .sort_values(by='tube_assembly_id')
                      .dropna()
                      .rename(columns={'value': 'component_id'}))

assembly_components.head(10)

,tube_assembly_id,variable,component_id
0,TA-00001,component_id_1,C-1622
21198,TA-00001,component_id_2,C-1629
1,TA-00002,component_id_1,C-1312
2,TA-00003,component_id_1,C-1312
3,TA-00004,component_id_1,C-1312
4,TA-00005,component_id_1,C-1624
42400,TA-00005,component_id_3,C-1641
21202,TA-00005,component_id_2,C-1631
5,TA-00006,component_id_1,C-1624
21203,TA-00006,component_id_2,C-1631


In [21]:
assembly_component_types = assembly_components.merge(components, how='left');
assembly_component_types.head(10)

,tube_assembly_id,variable,component_id,name,component_type_id
0,TA-00001,component_id_1,C-1622,NUT-SWIVEL,CP-025
1,TA-00001,component_id_2,C-1629,SLEEVE-ORFS,CP-024
2,TA-00002,component_id_1,C-1312,NUT-FLARED,CP-028
3,TA-00003,component_id_1,C-1312,NUT-FLARED,CP-028
4,TA-00004,component_id_1,C-1312,NUT-FLARED,CP-028
5,TA-00005,component_id_1,C-1624,NUT-SWIVEL,CP-025
6,TA-00005,component_id_3,C-1641,CONNECTOR-SEAL,CP-014
7,TA-00005,component_id_2,C-1631,SLEEVE-ORFS,CP-024
8,TA-00006,component_id_1,C-1624,NUT-SWIVEL,CP-025
9,TA-00006,component_id_2,C-1631,SLEEVE-ORFS,CP-024


In [25]:
table = pd.crosstab(assembly_component_types['tube_assembly_id'],
                   assembly_component_types['component_id']);
table.reset_index();
table.columns.name = '';
print(table.shape);
table.head()

(19149, 2048)


,9999,C-0001,C-0002,C-0003,C-0004,C-0005,C-0006,C-0007,C-0008,C-0009,...,C-2038,C-2039,C-2040,C-2041,C-2042,C-2043,C-2044,C-2045,C-2046,C-2047
tube_assembly_id,,,,,,,,,,,,,,,,,,,,,
TA-00001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TA-00002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TA-00003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TA-00004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TA-00005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
comp = pd.concat((pd.read_csv(path) for path in glob(os.path.join(SOURCE,'*.csv'))), sort=False);
columns = ['component_id', 'component_type_id', 'orientation', 'unique_feature', 'weight'];
comp = comp[columns];
comp['orientation'] = (comp['orientation'] == 'Yes').astype(int);
comp['unique_feature'] = (comp['unique_feature'] == 'Yes').astype(int);
comp['weight'] = comp['weight'].fillna(comp['weight'].median());

In [27]:
comp.head()

,component_id,component_type_id,orientation,unique_feature,weight
0,C-0007,CP-014,0,0,0.028
1,C-0030,CP-015,1,0,0.914
2,C-0041,CP-014,0,0,0.120
3,C-0043,CP-014,0,0,0.090
4,C-0044,CP-014,0,0,0.182


In [32]:
materials['components_total'] = sum(materials[f'quantity_{n}'].fillna(0) for n in range(1, 9));
materials['components_distinct'] = sum(materials[f'component_id_{n}'].notnull().astype(int) for n in range(1, 9));
materials['orientation'] = 0;
materials['unique_feature'] = 0;
materials['weight'] = 0;

for n in range(1, 9):
    materials = materials.merge(comp, how='left', left_on=f'component_id_{n}', right_on='component_id', suffixes=('', f'_{n}'));

for col in materials:
    if 'orientation' in col or 'unique_feature' in col or 'weight' in col:
        materials[col] = materials[col].fillna(0);

materials['orientation'] = sum(materials[f'orientation_{n}'] for i in range(1, 9));
materials['unique_feature'] = sum(materials[f'unique_feature_{n}'] for i in range(1, 9));
materials['weight'] = sum(materials[f'weight_{n}'] for i in range(1, 9));

materials.head()

KeyboardInterrupt: 